# testing metrics
this code test different metrics i.e. circularity etc to see if is doing the job it is supposed to.

using 30th july processed to run stats

also a lot of csv methods used

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from glob import glob
import seaborn as sns
import xesmf as xe
import pandas as pd
#import def_homebrew as hb ## homemade functions xox
from scipy.special import gamma
import netCDF4 as nc
from datetime import datetime, timedelta

import h5py ####
from PIL import Image
#from IPython.display import display #
#import cv2 # not working
import os
import shutil

from scipy.ndimage import convolve, label
from skimage.measure import regionprops, find_contours
from scipy.spatial import ConvexHull, distance_matrix
from skimage.morphology import remove_small_holes ## remove holes <3
from scipy.ndimage import binary_fill_holes
from skimage import measure
from cv2 import cvtColor, COLOR_BGR2GRAY, threshold, THRESH_BINARY, THRESH_OTSU
import random
import tensorflow as tf

2024-11-19 11:34:30.446503: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [23]:
#path = '/home/users/esree/processed_images/'
#

## image location ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
image_path = '/gws/nopw/j04/dcmex/users/ezriab/processed_images/2ds/ch_0/v4_2_220730153000/'
new_image_loc = '/home/users/esree/image-examples/v4/'
## stats location ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#csv_path = '/gws/nopw/j04/dcmex/users/ezriab/processed_images/2ds/ch_0/v3_220730153000/'
#csv_path = '/gws/nopw/j04/dcmex/users/ezriab/processed_images/2ds/ch_0/220730153000/'
csv_path = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/hvps/'

#csv = 'merged_stats_habits.csv'
#csv = '220730153000_stats_habits.csv'
#csv= 'v3_220730153000_stats_habits.csv'
csv = 'flight_220730152928.csv'

og_df = pd.read_csv(csv_path+csv)
## again with the new naming scheme in the df, everything is difficult, so are removing it
#og_df['name'] = og_df['name'].str.replace('_ch_0', '', regex=False)
# remove truncated ones
df = og_df[(og_df['first_diode_trunc'] == 0) & (og_df['last_diode_trunc'] == 0)]

In [24]:
diam = og_df['d_max']
smallest = min(diam)
smallest

1500.199278692033

In [9]:
trunc_df = og_df[(og_df['first_diode_trunc'] != 0) & (og_df['last_diode_trunc'] != 0)] # both trunc
df_or = og_df[(og_df['first_diode_trunc'] != 0) | (og_df['last_diode_trunc'] != 0)]

trunc_list = list(trunc_df['name'])

#new_image_path = '/gws/nopw/j04/dcmex/users/ezriab/processed_images/2ds/ch_0/220730153000/'
save_path = '/home/users/esree/image-examples/v3/trunc/'

for name in trunc_list:
    try:
        og_file = f'{path}{name}.png'
        if os.path.exists(og_file):
            print('y')
            shutil.copy(og_file, save_path)  # Copy the file
            print(f'{name} copied')
        else:
            print(f'{name} doesn\'t exist')
        
    except:
        print(f'{name} doesn\'t exist')

y
131385_30ch0 copied
y
135775_30ch0 copied
y
288472_30ch0 copied
y
290973_30ch0 copied
y
315352_30ch0 copied
y
316364_30ch0 copied
y
316915_30ch0 copied
y
318747_30ch0 copied
y
320298_30ch0 copied
y
321046_30ch0 copied
y
325593_30ch0 copied
y
326080_30ch0 copied
y
328059_30ch0 copied
y
337352_30ch0 copied
y
356856_30ch0 copied
y
409061_30ch0 copied
y
421760_30ch0 copied
y
444419_30ch0 copied
y
461074_30ch0 copied
y
462805_30ch0 copied
y
464692_30ch0 copied
y
519359_30ch0 copied
y
763517_30ch0 copied
y
763674_30ch0 copied
y
969672_30ch0 copied
y
998981_30ch0 copied
y
1002353_30ch0 copied
y
1031613_30ch0 copied


In [12]:
categories = ['CA', 'Co',  'CC', 'CBC', 'CG', 'HPC', 'Dif', 'FA', 'WD'] # match paper predictions

### make dict with each habit broken into a df
habit_dict_df = {} # empty dictionary, with each habit as key
for habit in categories:
    one_habit = df[(df['Category'] == habit)]
    habit_dict_df[habit] = one_habit  ## add to dict
    habit_count = one_habit['name'].count()
    print(f'{habit} {habit_count}')

## open specif habit: habit_dict_df['CA'] etc

CA 74
Co 152
CC 409
CBC 9
CG 1366
HPC 1017
Dif 447
FA 33
WD 110


### copying png images based on category 

In [13]:
## number of images to assess 
n_images = 20

## loop for moving particle images to somewhere more local to view + do assessments
for habit_class in categories:
    random_particles = []
    name_list = habit_dict_df[habit_class]['name'].tolist()
    new_loc = f'{new_image_loc}{habit_class}/'
    if not os.path.exists(new_loc):
        os.makedirs(new_loc)
    
    # Loop to generate random names
    while len(random_particles) < n_images:
        random_choice = random.choice(name_list)
        
        if random_choice in habit_dict_df[habit_class]['name'].values:
            og_file = f'{image_path}{random_choice}.png'

            if os.path.exists(new_loc) and os.path.exists(og_file):
                shutil.copy(og_file, new_loc)  # Copy the file
                random_particles.append(random_choice)

    
    print(f'{habit_class} done!')

CA done!
Co done!
CC done!
CBC done!
CG done!
HPC done!
Dif done!
FA done!
WD done!


In [ ]:
## a needs script further down to be ran before working - !! dic_image_names !!
image_names = list(dic_image_names['WD'])
new_image_path = '/gws/nopw/j04/dcmex/users/ezriab/processed_images/2ds/ch_0/220730153000/'
other_image_loc = '/home/users/esree/image-examples/WD-2/'

for name in image_names:
    try:
        og_file = f'{new_image_path}{name}.png'
        if os.path.exists(og_file):
            print('y')
            shutil.copy(og_file, other_image_loc)  # Copy the file
            print(f'{name} copied')
        else:
            print(f'{name} doesn\'t exist')
        
    except:
        print(f'{name} doesn\'t exist')


### open up stats about particles

In [ ]:
## list images that have been selected in the folder
new_image_loc = '/home/users/esree/image-examples/v2/'
categories = ['CA', 'Co',  'CC', 'CBC', 'CG', 'HPC', 'Dif', 'FA', 'WD'] # match paper predictions

path_list = []
for habit_class in categories:
    new_loc = f'{new_image_loc}{habit_class}/'
    path_list.append(new_loc)

## making dictionary of list of file names in the extracted images
dic_image_names = {}
for i in range(len(categories)):

    if os.path.exists(path_list[i]):
        # get string of full path + filenames in specif location
        file_list = glob(path_list[i]+'*.png')
        
        # just get file names
        file_names = []
        for file_path in file_list:
            base_name = os.path.basename(file_path)
            name_without_extension = os.path.splitext(base_name)[0] ## basically get rid of .png
            file_names.append(name_without_extension)
                
        dic_image_names[categories[i]] = file_names  ## add to dict

In [ ]:
## make a new dictionary from habit_dict_df of just images extracted
image_dict_df = {}
for habit in categories: #categories:
    image_list = dic_image_names[habit]
    sub_sample = habit_dict_df[habit][habit_dict_df[habit]['name'].isin(image_list)] # subsample df based on name crieteria
    sub_sample.set_index('name', inplace=True) ## quite important, the name is now the index - to pull out 
    image_dict_df[habit] = sub_sample


#habit_dict_df['CA']['name'].count()

In [ ]:
list(image_dict_df['CG'].columns.values)

In [ ]:
#print(image_dict_df['CG'])
#circularity_calc = np.divide((spec_region.perimeter**2),(4*np.pi*spec_region.area))
#row = image_dict_df['WD'].loc['37032_4']
row = og_df['name'].loc['172078_1']


#circularity = np.divide((row['perimeter']**2),(4*np.pi*row['area']))

#print(circularity)
print(row)

In [ ]:
#for row in image_dict_df['WD']:
#    print(row)
image_dict_df['WD']['circularity']

### subset the values

In [ ]:
df['circularity'] = np.floor(df['circularity'] * 10) / 10

In [ ]:
df['circularity']
all_li = df[(df['circularity'] <= 1.2)]
li_names = list(all_li['name'])

In [ ]:
new_image_path = '/gws/nopw/j04/dcmex/users/ezriab/processed_images/2ds/ch_0/220730153000/'
save_path = '/home/users/esree/image-examples/v2/WD/actual_circ/'

for name in li_names:
    try:
        og_file = f'{new_image_path}{name}.png'
        if os.path.exists(og_file):
            print('y')
            shutil.copy(og_file, save_path)  # Copy the file
            print(f'{name} copied')
        else:
            print(f'{name} doesn\'t exist')
        
    except:
        print(f'{name} doesn\'t exist')

## okay something is going wrong, need to check h5 files
### i am so fucked

In [ ]:
path = '/gws/nopw/j04/dcmex/users/ezriab/raw_h5/2ds/ch_0/'
# e.g. one file 
file = 'Export_base220730153000.h5'
# open file
f2ds = h5py.File(path+file,'r')


# structure of the file
#list(f2ds.keys()) ## ['ImageData', 'ImageTimes']

ds_image = f2ds['ImageData'] # shape (128, 200000) --> 128 pixels width, 200000 length
ds_time = f2ds['ImageTimes'] # shape (100000, 3) --> (rows, columns) related to time

# make time variable -> 3 seperate columns
og_t_xr = xr.Dataset({'ImageTimes': (('data', 'time_vars'),ds_time)})
sec_since = og_t_xr['ImageTimes'][:,0] # seconds since midnight UTC
pixel_slice = og_t_xr['ImageTimes'][:,1] # number of slices of pixel per image (contain -1, to fill)
# we can use pixel slice to correcly divide up data ^ the index to call f2ds['ImageData']
# these slices have been pre-determined by Jonny + algorithm he has written
### basically pixel slice gives a rough idea of particle location

bit_time = og_t_xr['ImageTimes'][:,2] # 32 bit (instrument things) - not useful

# do cumulative sum of pixel slices -> can use this as index to slice + extract single crystals
pix_sum = pixel_slice.cumsum(dim='data', dtype ='int')

# shove together into a useful xarray 
time_xr = xr.Dataset({
    'sec_since': sec_since,
    'pixel_slice': pixel_slice,
    'bit_time': bit_time,
    'pix_sum': pix_sum})

## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
## functions to make code run smoothly
def stats_description(bw_crystal, fill_hole_thresh):
    #take binary image, fill in small holes and returns object containing stats about crystal'
    
    filled_particle = remove_small_holes(bw_crystal.image, area_threshold=fill_hole_thresh) # fill in voids within binary image - better estimation of stats # may need to be altered
    
    # can see the filled in particle if needs be
    #plt.imshow(filled_particle, cmap='gray')
    
    if filled_particle.shape[0] < 2 or filled_particle.shape[1] < 2:
        return filled_particle, None
        
    contours = measure.find_contours(filled_particle, 0.5)
    if contours:
        contour = max(contours, key=lambda x: x.shape[0])  # Sort contours by area (largest first) and select the largest contour
        
        labeled_image = measure.label(filled_particle)  # Label the image based on the threshold
        region = measure.regionprops(labeled_image)[0]  # Assumes largest labeled region corresponds to largest contour
        
        return filled_particle, region
    else:
        return filled_particle, None
## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
## function to calculate truncation of particle
def calc_truncation(a_slice, particle):
    # the intial slice is the raw 2ds data - of whole array, and particle is one selected by regionprops (and has to be 1s + 0s)
    ## assume slices are small + don't contain too many odd bits
    # first convert to 0 and 1 for calculation of truncation
    alt_crystal = np.where(a_slice == 255, 0, 1) # i.e. 255 (blank area)=0, and where crystal is (was 0) = 1
    # so sum up number of particle pixels are on the edge of the slice
    first_diode = sum(alt_crystal[0,:]) 
    last_diode = sum(alt_crystal[-1,:])

    ## this calculates how many pixels are top / bottom of the particle + then infer number pixels touching
    top_particle = np.sum(particle[0] == 1)
    bottom_particle = np.sum(particle[-1] == 1)

    n_top, n_bottom = 0, 0  # Initialize variables, default 0 when conditions are not met

    # Top pixel touching logic
    if first_diode != 0 and first_diode >= top_particle:
        n_top = top_particle
    elif first_diode == 0:
        n_top = 0

    # Bottom pixel touching logic
    if last_diode != 0 and last_diode >= bottom_particle:
        n_bottom = bottom_particle
    elif last_diode == 0:
        n_bottom = 0

    return n_top, n_bottom # number pixels touching top / bottom respectively


In [ ]:
for particle in dic_image_names['WD']:
    print(particle)
    row = image_dict_df['WD'].loc[particle]
    
    circularity = np.divide((row['perimeter']**2),(4*np.pi*row['area']))

    im_s_idx = row['start_index']
    im_e_idx = row['end_index'] ## may want to add more - to look at a bigger slice
    
    one_crystal = f2ds['ImageData'][:,im_s_idx:im_e_idx] # extract 1 crystal
    
    ## this will plot the crystal
    plt.imshow(one_crystal, cmap='gray')
    plt.axis('off')  # Turn off axis labels
    plt.show()


In [ ]:

minimum_area = 15 # very quick metric to stop the processing of particles with area < 15 pixels

length_threshold = 100 #300 # mu - need this minimum length of max dimension to extract the particle
pixel_resolution = 10 # mu
desired_image_size = 200 # (assume we want a square image) 200 x 200

wd_names = list(dic_image_names['WD'])
one_crystal = wd_names[0]
print(one_crystal)
row = image_dict_df['WD'].loc[one_crystal]
    
circularity = np.divide((row['perimeter']**2),(4*np.pi*row['area']))

im_s_idx = row['start_index']
im_e_idx = row['end_index'] ## may want to add more - to look at a bigger slice

one_crystal = f2ds['ImageData'][:,im_s_idx:im_e_idx] # extract 1 crystal
#one_crystal = f2ds['ImageData'][:,im_s_idx-40:im_e_idx+100] # extract 1 crystal
'''
## this will plot the crystal
plt.imshow(one_crystal, cmap='gray')
plt.axis('off')  # Turn off axis labels
plt.show()
'''

binary_image = (one_crystal == 0)
labeled_image, num_features = label(binary_image)

props = regionprops(labeled_image)

for particle in props:
    filled_part, spec_region = stats_description(particle,5)
                    ### !!!!!! important part - this is where useful crystals are getting stats + being recorded in the dataframe
                    
    filled_part = filled_part.astype(np.float32) ## convert to float 0 and 1s
     
    if spec_region and spec_region.major_axis_length * pixel_resolution >= length_threshold and spec_region.minor_axis_length > 4 :
        print(f'{im_s_idx}_{particle.label}')
        plt.imshow(filled_part, cmap='gray')
        plt.axis('off')  # Turn off axis labels
        plt.show() 

        filled_part = np.expand_dims(filled_part, axis=-1) ## add extra dimention - this is for adding padding
        imagex = tf.image.resize_with_crop_or_pad(filled_part, desired_image_size, desired_image_size)
        
        plt.imshow(imagex, cmap='gray')
        plt.axis('off') # Turn off axis labels
        plt.show()

        if np.all(imagex == 0):
            print("The image is blank (all pixels are zero).")
        elif np.all(imagex == imagex[0, 0, 0]):
            print("The image is blank (all pixels have the same constant value).")
        else:
            print("The image contains varying pixel values.")

## okay, updated script testing
pull out images with circularity <1.2 

In [ ]:
## updated v2 script, pulling out li partilces + saving them
stats_csv = '/gws/nopw/j04/dcmex/users/ezriab/processed_images/2ds/ch_0/220730153000/flight_220730153000.csv'
new_df = pd.read_csv(stats_csv)

# make df of just li < 1.2
all_li = new_df[(new_df['circularity'] <= 1.2)]
name_list = all_li['name'].apply(lambda x: x[:-5]).tolist() ## get list of names of that satisfy the criteria (and remove last 5 values)
name_list

In [ ]:
new_image_path = '/gws/nopw/j04/dcmex/users/ezriab/processed_images/2ds/ch_0/220730153000/'
save_path = '/home/users/esree/image-examples/sample-LI/'

for name in name_list:
    try:
        og_file = f'{new_image_path}{name}.png'
        if os.path.exists(og_file):
            print('y')
            shutil.copy(og_file, save_path)  # Copy the file
            print(f'{name} copied')
        else:
            print(f'{name} doesn\'t exist')
        
    except:
        print(f'{name} doesn\'t exist')